In [2]:
import configparser
import os
from pathlib import Path
import pkgutil
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
import pandas as pd

In [3]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
dir_prj = Path.cwd().parent

dir_ref = dir_prj / 'references'

In [4]:
from arcgis.geoenrichment._business_analyst import BusinessAnalyst, Country

index_col_nm = 'index_name'

meta_excel_pth = dir_ref / 'index_variables.xlsx'

In [5]:
usa = Country(iso3='USA', source='local')

In [6]:
ev = usa.enrich_variables

ev

,name,alias,data_collection,enrich_name,enrich_field_name
0,THHBASE,2022 Tapestry Household Base,BabyProductsToysGames,BabyProductsToysGames.THHBASE,BabyProductsToysGames_THHBASE
1,TADULTBASE,2022 Tapestry Adult Pop Base,BabyProductsToysGames,BabyProductsToysGames.TADULTBASE,BabyProductsToysGames_TADULTBASE
2,MP03017h_B,2022 HH Owns Any Baby Furniture or Equipment,BabyProductsToysGames,BabyProductsToysGames.MP03017h_B,BabyProductsToysGames_MP03017h_B
3,MP03017h_I,2022 Index: HH Owns Any Baby Furniture or Equi...,BabyProductsToysGames,BabyProductsToysGames.MP03017h_I,BabyProductsToysGames_MP03017h_I
4,MP03018h_B,2022 HH Bought Baby Furniture or Equipment/6 Mo,BabyProductsToysGames,BabyProductsToysGames.MP03018h_B,BabyProductsToysGames_MP03018h_B
...,...,...,...,...,...
18913,MOEMEDYRMV,2020 Median Year Householder Moved In MOE (ACS...,yearmovedin,yearmovedin.MOEMEDYRMV,yearmovedin_MOEMEDYRMV
18914,RELMEDYRMV,2020 Median Year Householder Moved In REL (ACS...,yearmovedin,yearmovedin.RELMEDYRMV,yearmovedin_RELMEDYRMV
18915,ACSOWNER,2020 Owner Households (ACS 5-Yr),yearmovedin,yearmovedin.ACSOWNER,yearmovedin_ACSOWNER
18916,MOEOWNER,2020 Owner Households MOE (ACS 5-Yr),yearmovedin,yearmovedin.MOEOWNER,yearmovedin_MOEOWNER


## Meta-Variable Exploration

### Income

In [7]:
hhinc_df = ev[
    (ev.alias.str.lower().str.contains('income'))
    & (ev.name.str.endswith('_CY'))
    & (ev.name.str.startswith('HINC'))
].drop_duplicates('name').reset_index(drop=True)

hhinc_df[index_col_nm] = 'income'

hhinc_df

,name,alias,data_collection,enrich_name,enrich_field_name,index_name
0,HINC0_CY,2022 HH Income <$15000,Policy,Policy.HINC0_CY,Policy_HINC0_CY,income
1,HINC15_CY,2022 HH Income $15000-24999,Policy,Policy.HINC15_CY,Policy_HINC15_CY,income
2,HINC25_CY,2022 HH Income $25000-34999,Policy,Policy.HINC25_CY,Policy_HINC25_CY,income
3,HINC35_CY,2022 HH Income $35000-49999,Policy,Policy.HINC35_CY,Policy_HINC35_CY,income
4,HINC50_CY,2022 HH Income $50000-74999,Policy,Policy.HINC50_CY,Policy_HINC50_CY,income
5,HINC75_CY,2022 HH Income $75000-99999,Policy,Policy.HINC75_CY,Policy_HINC75_CY,income
6,HINC100_CY,2022 HH Income $100000-149999,Policy,Policy.HINC100_CY,Policy_HINC100_CY,income
7,HINC150_CY,2022 HH Income $150000-199999,Policy,Policy.HINC150_CY,Policy_HINC150_CY,income
8,HINC200_CY,2022 HH Income $200000+,Policy,Policy.HINC200_CY,Policy_HINC200_CY,income


### Household Size

In [8]:
hhsize_df = ev[
    (ev.alias.str.lower().str.contains('household'))
    & (ev.alias.str.lower().str.contains('size'))
].drop_duplicates('name').reset_index(drop=True)

hhsize_df[index_col_nm] = 'household_size'

hhsize_df

,name,alias,data_collection,enrich_name,enrich_field_name,index_name
0,AVGHHSZ_CY,2022 Average Household Size,AtRisk,AtRisk.AVGHHSZ_CY,AtRisk_AVGHHSZ_CY,household_size
1,AVGHHSZ_FY,2027 Average Household Size,householdtotals,householdtotals.AVGHHSZ_FY,householdtotals_AVGHHSZ_FY,household_size
2,AVGHHSZ20,2020 Average Household Size,householdtotals,householdtotals.AVGHHSZ20,householdtotals_AVGHHSZ20,household_size
3,AVGHHSZ10,2010 Average Household Size,householdtotals,householdtotals.AVGHHSZ10,householdtotals_AVGHHSZ10,household_size
4,AVGHHSZ00,2000 Average Household Size,householdtotals,householdtotals.AVGHHSZ00,householdtotals_AVGHHSZ00,household_size
5,AVGHHSZ,Average Household Size,KeyGlobalFacts,KeyGlobalFacts.AVGHHSZ,KeyGlobalFacts_AVGHHSZ,household_size
6,AUX2,Average Household Size,KeyGlobalFacts,KeyGlobalFacts.AUX2,KeyGlobalFacts_AUX2,household_size


### Home Value

In [9]:
hv_df = ev[
    (ev.alias.str.lower().str.contains('home'))
    & (ev.alias.str.lower().str.contains('value'))
    & (ev.name.str.endswith('_CY'))
    & (ev.name.str.startswith('VAL'))
    & (~ev.name.str.contains('BASE'))
].drop_duplicates('name').reset_index(drop=True)

hv_df[index_col_nm] = 'home_value'

hv_df

,name,alias,data_collection,enrich_name,enrich_field_name,index_name
0,VAL0_CY,2022 Home Value <$50000,homevalue,homevalue.VAL0_CY,homevalue_VAL0_CY,home_value
1,VAL50K_CY,2022 Home Value $50K-99999,homevalue,homevalue.VAL50K_CY,homevalue_VAL50K_CY,home_value
2,VAL100K_CY,2022 Home Value $100K-149999,homevalue,homevalue.VAL100K_CY,homevalue_VAL100K_CY,home_value
3,VAL150K_CY,2022 Home Value $150K-199999,homevalue,homevalue.VAL150K_CY,homevalue_VAL150K_CY,home_value
4,VAL200K_CY,2022 Home Value $200K-249999,homevalue,homevalue.VAL200K_CY,homevalue_VAL200K_CY,home_value
5,VAL250K_CY,2022 Home Value $250K-299999,homevalue,homevalue.VAL250K_CY,homevalue_VAL250K_CY,home_value
6,VAL300K_CY,2022 Home Value $300K-399999,homevalue,homevalue.VAL300K_CY,homevalue_VAL300K_CY,home_value
7,VAL400K_CY,2022 Home Value $400K-499999,homevalue,homevalue.VAL400K_CY,homevalue_VAL400K_CY,home_value
8,VAL500K_CY,2022 Home Value $500K-749999,homevalue,homevalue.VAL500K_CY,homevalue_VAL500K_CY,home_value
9,VAL750K_CY,2022 Home Value $750K-999999,homevalue,homevalue.VAL750K_CY,homevalue_VAL750K_CY,home_value


In [10]:
list(hv_df.name)

['VAL0_CY',
 'VAL50K_CY',
 'VAL100K_CY',
 'VAL150K_CY',
 'VAL200K_CY',
 'VAL250K_CY',
 'VAL300K_CY',
 'VAL400K_CY',
 'VAL500K_CY',
 'VAL750K_CY',
 'VAL1M_CY',
 'VAL2M_CY']

### Wealth

In [11]:
wealth_df = ev[
    (ev.data_collection.str.lower().str.contains('wealth'))
    & (ev.name.str.startswith('NW'))
    & (~ev.name.str.startswith('NWBASE'))
].drop_duplicates('name').reset_index(drop=True)

wealth_df[index_col_nm] = 'wealth'

wealth_df

,name,alias,data_collection,enrich_name,enrich_field_name,index_name
0,NW0_CY,2022 Net Worth <$15000,Wealth,Wealth.NW0_CY,Wealth_NW0_CY,wealth
1,NW15_CY,2022 Net Worth $15000-$34999,Wealth,Wealth.NW15_CY,Wealth_NW15_CY,wealth
2,NW35_CY,2022 Net Worth $35000-$49999,Wealth,Wealth.NW35_CY,Wealth_NW35_CY,wealth
3,NW50_CY,2022 Net Worth $50000-$74999,Wealth,Wealth.NW50_CY,Wealth_NW50_CY,wealth
4,NW75_CY,2022 Net Worth $75000-$99999,Wealth,Wealth.NW75_CY,Wealth_NW75_CY,wealth
5,NW100_CY,2022 Net Worth $100000-$149999,Wealth,Wealth.NW100_CY,Wealth_NW100_CY,wealth
6,NW150_CY,2022 Net Worth $150000-$249999,Wealth,Wealth.NW150_CY,Wealth_NW150_CY,wealth
7,NW250_CY,2022 Net Worth $250000-$499999,Wealth,Wealth.NW250_CY,Wealth_NW250_CY,wealth
8,NW500_CY,2022 Net Worth $500000-$999999,Wealth,Wealth.NW500_CY,Wealth_NW500_CY,wealth


In [12]:
list(wealth_df.name)

['NW0_CY',
 'NW15_CY',
 'NW35_CY',
 'NW50_CY',
 'NW75_CY',
 'NW100_CY',
 'NW150_CY',
 'NW250_CY',
 'NW500_CY']

### Age

In [13]:
age_df = ev[
    (ev.alias.str.lower().str.contains(' age '))
    & (ev.name.str.startswith('POP'))
    & (ev.name.str.endswith('CY'))
    & (~ev.name.str.contains('BASE'))
    & (~ev.name.str.contains('UP'))
].drop_duplicates('name').reset_index(drop=True)

age_df[index_col_nm] = 'age'

age_df

,name,alias,data_collection,enrich_name,enrich_field_name,index_name
0,POP0_CY,2022 Population Age 0-4,5yearincrements,5yearincrements.POP0_CY,F5yearincrements_POP0_CY,age
1,POP5_CY,2022 Population Age 5-9,5yearincrements,5yearincrements.POP5_CY,F5yearincrements_POP5_CY,age
2,POP10_CY,2022 Population Age 10-14,5yearincrements,5yearincrements.POP10_CY,F5yearincrements_POP10_CY,age
3,POP15_CY,2022 Population Age 15-19,5yearincrements,5yearincrements.POP15_CY,F5yearincrements_POP15_CY,age
4,POP20_CY,2022 Population Age 20-24,5yearincrements,5yearincrements.POP20_CY,F5yearincrements_POP20_CY,age
5,POP25_CY,2022 Population Age 25-29,5yearincrements,5yearincrements.POP25_CY,F5yearincrements_POP25_CY,age
6,POP30_CY,2022 Population Age 30-34,5yearincrements,5yearincrements.POP30_CY,F5yearincrements_POP30_CY,age
7,POP35_CY,2022 Population Age 35-39,5yearincrements,5yearincrements.POP35_CY,F5yearincrements_POP35_CY,age
8,POP40_CY,2022 Population Age 40-44,5yearincrements,5yearincrements.POP40_CY,F5yearincrements_POP40_CY,age
9,POP45_CY,2022 Population Age 45-49,5yearincrements,5yearincrements.POP45_CY,F5yearincrements_POP45_CY,age


In [14]:
list(age_df.name)

['POP0_CY',
 'POP5_CY',
 'POP10_CY',
 'POP15_CY',
 'POP20_CY',
 'POP25_CY',
 'POP30_CY',
 'POP35_CY',
 'POP40_CY',
 'POP45_CY',
 'POP50_CY',
 'POP55_CY',
 'POP60_CY',
 'POP65_CY',
 'POP70_CY',
 'POP75_CY',
 'POP80_CY',
 'POP85_CY']

### Home Age

In [15]:
home_age_df = ev[
    (ev.name.str.startswith('ACSBLT'))
].drop_duplicates('name').reset_index(drop=True)

home_age_df[index_col_nm] = 'home_age'

home_age_df

,name,alias,data_collection,enrich_name,enrich_field_name,index_name
0,ACSBLT2014,2020 HUs/Year Built: 2014/Later (ACS 5-Yr),yearbuilt,yearbuilt.ACSBLT2014,yearbuilt_ACSBLT2014,home_age
1,ACSBLT2010,2020 HUs/Year Built: 2010-2013 (ACS 5-Yr),yearbuilt,yearbuilt.ACSBLT2010,yearbuilt_ACSBLT2010,home_age
2,ACSBLT2000,2020 HUs/Year Built: 2000-2009 (ACS 5-Yr),yearbuilt,yearbuilt.ACSBLT2000,yearbuilt_ACSBLT2000,home_age
3,ACSBLT1990,2020 HUs/Year Built: 1990-1999 (ACS 5-Yr),yearbuilt,yearbuilt.ACSBLT1990,yearbuilt_ACSBLT1990,home_age
4,ACSBLT1980,2020 HUs/Year Built: 1980-1989 (ACS 5-Yr),yearbuilt,yearbuilt.ACSBLT1980,yearbuilt_ACSBLT1980,home_age
5,ACSBLT1970,2020 HUs/Year Built: 1970-1979 (ACS 5-Yr),yearbuilt,yearbuilt.ACSBLT1970,yearbuilt_ACSBLT1970,home_age
6,ACSBLT1960,2020 HUs/Year Built: 1960-1969 (ACS 5-Yr),yearbuilt,yearbuilt.ACSBLT1960,yearbuilt_ACSBLT1960,home_age
7,ACSBLT1950,2020 HUs/Year Built: 1950-1959 (ACS 5-Yr),yearbuilt,yearbuilt.ACSBLT1950,yearbuilt_ACSBLT1950,home_age
8,ACSBLT1940,2020 HUs/Year Built: 1940-1949 (ACS 5-Yr),yearbuilt,yearbuilt.ACSBLT1940,yearbuilt_ACSBLT1940,home_age
9,ACSBLT1939,2020 HUs/Year Built: 1939 or Earlier (ACS 5-Yr),yearbuilt,yearbuilt.ACSBLT1939,yearbuilt_ACSBLT1939,home_age


In [16]:
list(home_age_df.name)

['ACSBLT2014',
 'ACSBLT2010',
 'ACSBLT2000',
 'ACSBLT1990',
 'ACSBLT1980',
 'ACSBLT1970',
 'ACSBLT1960',
 'ACSBLT1950',
 'ACSBLT1940',
 'ACSBLT1939']

### Employment Diversity

In [17]:
empl_div_df = ev[
    (ev.alias.str.lower().str.contains('industry'))
    & (~ev.name.str.startswith('INDBASE'))
].drop_duplicates('name').reset_index(drop=True)

empl_div_df[index_col_nm] = 'employment_diversity'

empl_div_df

,name,alias,data_collection,enrich_name,enrich_field_name,index_name
0,INDAGRI_CY,2022 Industry: Agriculture,industry,industry.INDAGRI_CY,industry_INDAGRI_CY,employment_diversity
1,INDMIN_CY,2022 Industry: Mining,industry,industry.INDMIN_CY,industry_INDMIN_CY,employment_diversity
2,INDCONS_CY,2022 Industry: Construction,industry,industry.INDCONS_CY,industry_INDCONS_CY,employment_diversity
3,INDMANU_CY,2022 Industry: Manufacturing,industry,industry.INDMANU_CY,industry_INDMANU_CY,employment_diversity
4,INDWHTR_CY,2022 Industry: Wholesale Trade,industry,industry.INDWHTR_CY,industry_INDWHTR_CY,employment_diversity
5,INDRTTR_CY,2022 Industry: Retail Trade,industry,industry.INDRTTR_CY,industry_INDRTTR_CY,employment_diversity
6,INDTRAN_CY,2022 Industry: Transportation,industry,industry.INDTRAN_CY,industry_INDTRAN_CY,employment_diversity
7,INDUTIL_CY,2022 Industry: Utilities,industry,industry.INDUTIL_CY,industry_INDUTIL_CY,employment_diversity
8,INDINFO_CY,2022 Industry: Information,industry,industry.INDINFO_CY,industry_INDINFO_CY,employment_diversity
9,INDFIN_CY,2022 Industry: Finance/Insurance,industry,industry.INDFIN_CY,industry_INDFIN_CY,employment_diversity


In [18]:
list(empl_div_df.name)

['INDAGRI_CY',
 'INDMIN_CY',
 'INDCONS_CY',
 'INDMANU_CY',
 'INDWHTR_CY',
 'INDRTTR_CY',
 'INDTRAN_CY',
 'INDUTIL_CY',
 'INDINFO_CY',
 'INDFIN_CY',
 'INDRE_CY',
 'INDTECH_CY',
 'INDMGMT_CY',
 'INDADMN_CY',
 'INDEDUC_CY',
 'INDHLTH_CY',
 'INDARTS_CY',
 'INDFOOD_CY',
 'INDOTSV_CY',
 'INDPUBL_CY']

### Housing Diversity

In [19]:
hd_df = ev[
    (ev.name.str.startswith('ACS'))
    & (ev.data_collection == 'unitsinstructure')
].drop_duplicates('name').reset_index(drop=True)

hd_df[index_col_nm] = 'housing_diversity'

hd_df

,name,alias,data_collection,enrich_name,enrich_field_name,index_name
0,ACSUNT1DET,2020 Housing: 1 Detached Unit in Structure (AC...,unitsinstructure,unitsinstructure.ACSUNT1DET,unitsinstructure_ACSUNT1DET,housing_diversity
1,ACSUNT1ATT,2020 Housing: 1 Attached Unit in Structure (AC...,unitsinstructure,unitsinstructure.ACSUNT1ATT,unitsinstructure_ACSUNT1ATT,housing_diversity
2,ACSUNT2,2020 Housing: 2 Units in Structure (ACS 5-Yr),unitsinstructure,unitsinstructure.ACSUNT2,unitsinstructure_ACSUNT2,housing_diversity
3,ACSUNT3,2020 Housing: 3 or 4 Units in Structure (ACS 5...,unitsinstructure,unitsinstructure.ACSUNT3,unitsinstructure_ACSUNT3,housing_diversity
4,ACSUNT5,2020 Housing: 5 to 9 Units in Structure (ACS 5...,unitsinstructure,unitsinstructure.ACSUNT5,unitsinstructure_ACSUNT5,housing_diversity
5,ACSUNT10,2020 Housing: 10 to 19 Units in Structure (ACS...,unitsinstructure,unitsinstructure.ACSUNT10,unitsinstructure_ACSUNT10,housing_diversity
6,ACSUNT20,2020 Housing: 20 to 49 Units in Structure (ACS...,unitsinstructure,unitsinstructure.ACSUNT20,unitsinstructure_ACSUNT20,housing_diversity
7,ACSUNT50UP,2020 Housing: 50+ Units in Structure (ACS 5-Yr),unitsinstructure,unitsinstructure.ACSUNT50UP,unitsinstructure_ACSUNT50UP,housing_diversity
8,ACSUNTMOB,2020 Housing: Mobile Homes (ACS 5-Yr),unitsinstructure,unitsinstructure.ACSUNTMOB,unitsinstructure_ACSUNTMOB,housing_diversity
9,ACSUNTOTH,2020 Housing: Boat/RV/Van/etc. (ACS 5-Yr),unitsinstructure,unitsinstructure.ACSUNTOTH,unitsinstructure_ACSUNTOTH,housing_diversity


In [20]:
list(hd_df.name)

['ACSUNT1DET',
 'ACSUNT1ATT',
 'ACSUNT2',
 'ACSUNT3',
 'ACSUNT5',
 'ACSUNT10',
 'ACSUNT20',
 'ACSUNT50UP',
 'ACSUNTMOB',
 'ACSUNTOTH',
 'ACSTOTHU']

## Create Meta-DataFrame

In [21]:
meta_df = pd.concat([
    hhinc_df,
    hhsize_df,
    hv_df,
    wealth_df,
    age_df,
    home_age_df,
    empl_div_df,
    hd_df
])

In [22]:
meta_df.to_excel(meta_excel_pth, index=False)

In [23]:
index_nm_lst = list(meta_df[index_col_nm].unique())

index_nm_lst

['income',
 'household_size',
 'home_value',
 'wealth',
 'age',
 'home_age',
 'employment_diversity',
 'housing_diversity']

In [27]:
import json

print(json.dumps({idx_nm: list(meta_df[meta_df[index_col_nm] == idx_nm]['name']) for idx_nm in index_nm_lst}, indent=4))

{
    "income": [
        "HINC0_CY",
        "HINC15_CY",
        "HINC25_CY",
        "HINC35_CY",
        "HINC50_CY",
        "HINC75_CY",
        "HINC100_CY",
        "HINC150_CY",
        "HINC200_CY"
    ],
    "household_size": [
        "AVGHHSZ_CY",
        "AVGHHSZ_FY",
        "AVGHHSZ20",
        "AVGHHSZ10",
        "AVGHHSZ00",
        "AVGHHSZ",
        "AUX2"
    ],
    "home_value": [
        "VAL0_CY",
        "VAL50K_CY",
        "VAL100K_CY",
        "VAL150K_CY",
        "VAL200K_CY",
        "VAL250K_CY",
        "VAL300K_CY",
        "VAL400K_CY",
        "VAL500K_CY",
        "VAL750K_CY",
        "VAL1M_CY",
        "VAL2M_CY"
    ],
    "wealth": [
        "NW0_CY",
        "NW15_CY",
        "NW35_CY",
        "NW50_CY",
        "NW75_CY",
        "NW100_CY",
        "NW150_CY",
        "NW250_CY",
        "NW500_CY"
    ],
    "age": [
        "POP0_CY",
        "POP5_CY",
        "POP10_CY",
        "POP15_CY",
        "POP20_CY",
        "POP25_CY",
    